In [1]:
import os
import fnmatch
import numpy as np
import pandas as pd

In [2]:
repo = os.getcwd()
subjDirs = []
for subjDir in os.listdir(repo):
    if os.path.isdir(subjDir) & (fnmatch.fnmatch(subjDir, 'GroupID')):
        subjDirs.append(subjDir)

In [7]:
consMatD = np.empty([333,333])
for subj in sorted(subjDirs):
    consMat = np.genfromtxt('consMats/consMat_' + subj + '.csv', delimiter=' ')
    if subj == 'Subj01':
        consMatD = consMat
    else:
        consMatD = np.dstack((consMatD, consMat))

consMatDM = consMatD.mean(axis=2)
np.savetxt('consMatDM_low.csv', np.tril(consMatDM, k=-1), delimiter=' ')
## next, run cortmod_thrConn.m to threshold across ranges (0.15 to 0.01 by 0.001)

In [ ]:
## Updating the group-level cortical module networks

In [4]:
def zeropad(n,zeros=3):
    "Pad number n with zeros. Example: zeropad(7,3) == '007'"
    nstr = str(n)
    while len(nstr) < zeros:
        nstr = "0" + nstr
    return nstr

In [5]:
## set the module output (thresholded at 15%)
# initial module partition
mod_init = np.genfromtxt('links_group/modOut/clink_thr150.clu', delimiter=None)
df_mod_init = pd.DataFrame(mod_init, columns=['node_ID', 'module', 'flow'])

In [6]:
## iteratively update the module output from threshold of 0.15 to 0.01 by 0.001
for thr in reversed(range(10,150)):
    
    # load the modularity data (node ID, module, flow) of all tresholded connectivity matrix
    linkFiles = 'clink_thr' + str(zeropad(thr)) + '.clu'
    mod = np.genfromtxt('links_group/modOut/' + linkFiles, delimiter=None)
    df_mod = pd.DataFrame(mod, columns=['node_ID', 'module', 'flow'])
    
    # find modules consisting of at least five ROIs
    ms = []
    for m in list(set(mod[:,1])):
        if np.count_nonzero(mod[:,1] == m) >= 5:
            ms.append(m)
    df_mod_surv = df_mod[df_mod['module'].isin(ms)]
    
    # update the modularity output
    for roi in df_mod_surv['node_ID'].values:
        roi_mask = df_mod_surv.isin([roi])
        row_roi = list(roi_mask['node_ID'][roi_mask['node_ID'] == True].index)
        roi_mask0 = df_mod_init.isin([roi])
        row_roi0 = list(roi_mask0['node_ID'][roi_mask0['node_ID'] == True].index)
        df_mod_init.at[row_roi0, 'module'] = df_mod_surv.loc[row_roi, 'module'].values

df_mod_init.to_csv('links_group/modOut/cortmod_grp.csv', index=False)